<a href="https://colab.research.google.com/github/dvschultz/ml-art-colabs/blob/master/Colorization_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colorization Transformer

Source code accompanying the paper [Colorization Transformer](https://openreview.net/forum?id=5NA1PinlGFu) to be presented at
ICLR 2021.
Work by Manoj Kumar, Dirk Weissenborn and Nal Kalchbrenner.


<img src="https://github.com/google-research/google-research/raw/master/coltran/coltran_images.png" alt="Model images" width="1000"/>

### Important: Note on Evaluation:

Colorization is a multi-modal problem, where a given object can have multiple plausible colors.
On recolorizing a RGB image, a single random sample represents another plausible colorization. It
is unlikely to have the same colors as the ground-truth RGB image. So while evaluating the Colorization Transformer,
one should use distribution-level metrics such as the FID and NOT per-image metrics such as LPIPS or SSIM.


## Paper summary


ColTran consists of three components, a `colorizer`, `color upsampler` and `spatial upsampler`.

<img src="https://github.com/google-research/google-research/raw/master/coltran/coltran.png" alt="Model figure" width="800"/>

* The `colorizer` is an autoregressive, self-attention based architecture comprised of conditional transformer layers. It coarsely colorizes low resolution `64x64` grayscale images pixel-by-pixel.
* The `color upsampler` is a parallel, deterministic self-attention based network. It refines a coarse low resolution image
into a `64x64` RGB image.
* The architecture of the `spatial upsampler` is similar to the `color upsampler`. It superesolves the low resolution RGB image into the final output.
* The `colorizer` has an auxiliary parallel color prediction model composed of a single linear layer.

We report results after training each of these individual components on `4x4 TPUv2` chips. Please adjust the model size and batch-size while training using fewer resources. Our results on training these components using lower resources are available in the [appendix](https://openreview.net/pdf?id=5NA1PinlGFu).

Full configurations used to train models in the paper are available in the directory `configs`. Configs for extremely small models are provided at `test_configs` to test that the model build quickly. Set the flag `--steps_per_summaries=100` to output logs quickly. When sampling, set `config.sample.log_dir` to an
appropriate write directory.

In [ ]:
!nvidia-smi -L

# Setup

In [ ]:
#@title Clone repo and install dependencies
%cd /content/
!git clone --recurse-submodules https://github.com/google-research/google-research.git
%cd google-research/coltran/
!pip install -r requirements.txt

In [ ]:
#@title Download pre-trained model
!wget https://storage.googleapis.com/gresearch/coltran/coltran.zip -O coltran.zip
!unzip coltran.zip -d ./logdir

# Image colorisation

In [ ]:
#@title Set environment variables
 
import os
 
os.environ["LOGDIR"] = "/content/google-research/coltran/logdir/coltran"
!mkdir -p /content/input/ /content/output/
os.environ['IMG_DIR'] = "/content/input" 
os.environ['STORE_DIR'] = "/content/output"
 
mode = "colorize" #@param ["recolorize", "colorize"]
os.environ['MODE'] = mode

In [ ]:
#@title Run the colorizer to get a coarsely colorized image.
%cd /content/google-research/
!rm -rf $IMG_DIR/.ipynb_checkpoints/
!python -m coltran.custom_colorize --config=coltran/configs/colorizer.py --logdir=$LOGDIR/colorizer --img_dir=$IMG_DIR --store_dir=$STORE_DIR --mode=$MODE

In [ ]:
#@title Run the color upsampler to upsample the coarsely colored image.
%cd /content/google-research
!rm -rf $IMG_DIR/.ipynb_checkpoints/ $STORE_DIR/stage1/.ipynb_checkpoints
!python -m coltran.custom_colorize --config=coltran/configs/color_upsampler.py --logdir=$LOGDIR/color_upsampler --img_dir=$IMG_DIR --store_dir=$STORE_DIR --gen_data_dir=$STORE_DIR/stage1 --mode=$MODE

In [ ]:
#@title Run the spatial upsampler to super-resolve into the final output.
%cd /content/google-research/
!rm -rf $IMG_DIR/.ipynb_checkpoints/ $STORE_DIR/stage2/.ipynb_checkpoints
!python -m coltran.custom_colorize --config=coltran/configs/spatial_upsampler.py --logdir=$LOGDIR/spatial_upsampler --img_dir=$IMG_DIR --store_dir=$STORE_DIR --gen_data_dir=$STORE_DIR/stage2 --mode=$MODE